In [2]:
import requests
import pandas as pd
import pickle
from urllib.parse import urlencode
from datetime import datetime

### <span style="color:Red"> **1. Carrega Dados via mock API Pública** </span>

In [3]:
dataHoje = datetime.now().strftime("%d/%m/%Y")
parametros = {'dat_inicio': dataHoje, 'dat_fim': dataHoje}

baseurl = f'http://127.0.0.1:8000/api/v1/insumodou'

print(f'Recuperando dados da API - lista de do Diário Oficial para o dia {dataHoje}')
print()

response = requests.get(baseurl, params=parametros)

if response.status_code == requests.codes.ok:
    # Monta dataframe
    df_response =  pd.DataFrame.from_dict(response.json())

    print(f"{df_response.shape[0]} registros recuperados")
    print()
else:
    response.raise_for_status()

Recuperando dados da API - lista de do Diário Oficial para o dia 08/09/2020

214 registros recuperados



### <span style="color:Red">** 2. Ajusta dataframe com features de interesse ** </span>

In [4]:
df_response.drop(df_response.columns.difference(['id', 'dsc_texto']), 1, inplace=True)

### <span style="color:Red"> **3. Limpa tags HTML e torna texto minúsculo**</span>

In [5]:
    from bleach.sanitizer import Cleaner
    
    cleaner = Cleaner(tags=[], attributes={}, styles=[], protocols=[], strip=True, strip_comments=True, filters=None)
    
    # Retira todas as tags html do texto do DOU
    df_response['dsc_texto'] = df_response['dsc_texto'].apply(lambda x: cleaner.clean(x))

    print("Retiradas tags HTML")

Retiradas tags HTML


In [6]:
# Limpeza de ruído - Transformar em minúsculo, limpeza de caracteres especiais

import regex as re
import string

def limpa_ruido_texto(texto):
    texto = texto.lower()
    texto = re.sub('[%s]' % re.escape(string.punctuation), '', texto) # Retira pontuações
    texto = re.sub('\w*\d\w*', '', texto) # Retira números entre palavras
    texto = re.sub('§–', '', texto) # Casos específicos para os textos do DOU
    return texto

df_response['dsc_texto'] = df_response['dsc_texto'].apply(lambda x: limpa_ruido_texto(x))

df_response['dsc_texto'].head()

0    extrato de contratopregão presencial n° pp  co...
1    aviso de edital para alienação de bens leilão ...
2    despachofase de requerimento de pesquisaindefe...
3    portaria nº  de  de setembro de  secretário na...
4    portaria nº  de  de setembro de  ministro de e...
Name: dsc_texto, dtype: object

### <span style="color:Red"> **3.Remove linhas duplicadas no Dataframe**</span>

In [7]:
df_response = df_response.drop_duplicates()

print(f"Retiradas duplicações. {df_response.shape[0]} restantes")
print()

Retiradas duplicações. 214 restantes



### <span style="color:Red"> **4. Chama API com modelo para classificação e persistência**</span>

In [8]:
import json
import re

print("Iniciando a classificação e atualização dos registros")
print()

for registro in df_response.to_dict(orient='records'):
    
    baseurlupdate = f'http://127.0.0.1:8000/api/v1/insumodou/{registro["id"]}/'
  
    response = requests.patch(baseurlupdate, json = { "dsc_texto":registro["dsc_texto"]})

    if response.status_code == requests.codes.ok:
        print(f"{response.json().get('msg')}: Texto de id {response.json().get('id')} atualizado, resultado: {response.json().get('cod_interesse')}")
    else:
        print(f"Chamada a API para o id {response.json().get('id')} com erro {response.status_code}")


Iniciando a classificação e atualização dos registros

Classificação realizada: Texto de id 16129361 atualizado, resultado: Sem Interesse
Classificação realizada: Texto de id 16129400 atualizado, resultado: Sem Interesse
Classificação realizada: Texto de id 16129685 atualizado, resultado: Sem Interesse
Classificação realizada: Texto de id 16129694 atualizado, resultado: Sem Interesse
Classificação realizada: Texto de id 16129718 atualizado, resultado: Sem Interesse
Classificação realizada: Texto de id 16129724 atualizado, resultado: Sem Interesse
Classificação realizada: Texto de id 16129781 atualizado, resultado: Sem Interesse
Classificação realizada: Texto de id 16129814 atualizado, resultado: Sem Interesse
Classificação realizada: Texto de id 16129871 atualizado, resultado: Sem Interesse
Classificação realizada: Texto de id 16129892 atualizado, resultado: Sem Interesse
Classificação realizada: Texto de id 16129934 atualizado, resultado: De Interesse
Classificação realizada: Texto de